In [1]:
import xmltodict
from urllib.request import urlopen
import xml.etree.ElementTree as ET
import pandas as pd

from dateutil.parser import parse
from tqdm import tqdm

import matplotlib.pyplot as plt
from IPython.display import display 
from joblib import Parallel, delayed
import multiprocessing

import urllib
from bs4 import BeautifulSoup

import spacy
import json

nlp = spacy.load('en_core_web_sm')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
import string
import re
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

import pandas as pd
import numpy as np
import seaborn as sns
from multiprocessing import Pool

    
plt.style.use('seaborn-paper')
%matplotlib inline

In [2]:
link = 'http://clerk.house.gov/evs/{session}/roll{roll}.xml'

link_list = []

congress_session  = {'2018' : 129,
                     '2017' : 710,
                     '2016' : 622,
                     '2015' : 705, 
                     '2014' : 564,
                     '2013' : 641,
                     '2012' : 659,
                     '2011' : 949,
                     '2010' : 664,
                     '2009' : 991,
                     '2008' : 690,
                     '2007' : 1186,
                     '2006' : 543,
                     '2005' : 671,
                     '2004' : 544,
                     '2003' : 677,
                     '2002' : 484,
                     '2001' : 512,
                     '2000' : 603
                    }

for session, roll  in congress_session.items():
#     print(session, roll)
    for i in range (1, roll + 1):
        temp = link.replace('{session}', session)
        temp = temp.replace('{roll}', '{:03}'.format(i))
        link_list.append(temp)

print('link_list', len(link_list))
print(link_list[1])

link_list 12544
http://clerk.house.gov/evs/2004/roll002.xml


In [ ]:
#         <majority>D</majority>
#         <congress>103</congress>
#         <session>1st</session>
#         <chamber>U.S. House of Representatives</chamber>
#         <rollcall-num>615</rollcall-num>
#         <legis-num>H R 3167</legis-num>
#         <vote-question>On Agreeing to the Conference Report</vote-question>
#         <vote-type>RECORDED VOTE</vote-type>
#         <vote-result>Passed</vote-result>
#         <action-date>22-Nov-1993</action-date>
#         <action-time time-etz="23:59">11:59 PM</action-time>
#         <vote-desc>UNEMPLOYMENT COMPENSATION...</vote-desc>

#         <recorded-vote>
#             <legislator party="D" state="HI" role="legislator">Abercrombie</legislator>
#             <vote>Aye</vote>
#         </recorded-vote>

In [ ]:
def xml2df(xml_data):
#     print(xml_data)
    root = ET.XML(xml_data) # element tree
#     print(root)
    vote_metadata = root.find('vote-metadata')
    vote_data = root.find('vote-data').iter('recorded-vote')
    records = []
    
    for vote in vote_data:
        record = {} #Place holder for our record
        record['majority'] = vote_metadata.find('majority').text
        record['congress'] = vote_metadata.find('congress').text
        record['session'] = vote_metadata.find('session').text

        try:
            record['chamber'] = vote_metadata.find('chamber').text
        except Exception:
            record['chamber'] = None

        record['rollcall_num'] = vote_metadata.find('rollcall-num').text
        try:
            record['legis_num'] = vote_metadata.find('legis-num').text
        except Exception:
            record['legis_num'] = None

        record['vote_question'] = vote_metadata.find('vote-question').text
        record['vote_type'] = vote_metadata.find('vote-type').text
        record['vote_result'] = vote_metadata.find('vote-result').text
        record['action_date'] = vote_metadata.find('action-date').text
        record['action_time'] = vote_metadata.find('action-time').text
        record['vote_desc'] = vote_metadata.find('vote-desc').text
    
        record['party'] = vote.find('legislator').attrib['party']
        record['state'] = vote.find('legislator').attrib['state']
        record['role'] = vote.find('legislator').attrib['role']
        record['name'] = vote.find('legislator').text
        
        if any(vote.find('vote').text in s for s in ['Yea', 'Aye']):
            record['vote'] = 'Yea'
        elif any(vote.find('vote').text in s for s in ['No', 'Nay']):
            record['vote'] = 'Nay'
        else:
            continue
        records.append(record)
        
    return pd.DataFrame(records)
    
    
# xml = urlopen(link_list[2]).read()
# df = xml2df(xml)
# df.tail()

In [ ]:
%%time


def processLink(link):
#     print('processing', link)
    urlContent = urlopen(link)
    xml = urlContent.read()
    return xml2df(xml)

num_cores = multiprocessing.cpu_count()
print('num_cores', num_cores)

# processLink(link_list[2])
    
results = Parallel(n_jobs=num_cores)(delayed(processLink)(i) for i in tqdm(link_list))

df_votes = pd.concat(results)
df_votes.reset_index(drop=True, inplace=True)

# df_hansard['date'] =  pd.to_datetime(df_hansard['date'])

df_votes.to_csv('../data/votes_all.csv')
print(len(df_votes))

df_votes.tail()

In [ ]:
# df_votes = df_votes[df_votes['vote'].isin(['Yea', 'Aye', 'No', 'Nay'])]
df_votes['vote'].value_counts().plot(kind='bar', alpha=.5, figsize=(12, 6), fontsize=14)
df_votes.describe()

# House bills

In [ ]:
df_votes = pd.read_csv('../data/votes_all.csv')
df_filtered = df_votes.drop_duplicates('legis_num', keep='first')
df_filtered.reset_index(inplace=True)
df_filtered.tail()

In [ ]:

import re
link = 'https://www.congress.gov/bill/{congress}th-congress/house-bill/{billNum}/text?format=txt&r=1'
# 'https://www.congress.gov/bill/112th-congress/house-bill/hr2767/text?format=txt&r=1'

def creat_list(row):
#     print(row)
    link_temp = link.replace('{congress}', str(row['congress']))
    return link_temp.replace('{billNum}', ''.join(re.findall('\d+', row['legis_num'])))


df_filtered['link'] = df_filtered.apply(creat_list, axis=1)
df_filtered.tail()

In [ ]:


STOPLIST = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS))
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”", ' ', '', '``', '--', '''''',]

def tokenizeText(text):
#     print('hereeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', text)
    tokens = []
    for tok in nlp(text):
        tokens.append(tok.text.strip())
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    tokens = [tok for tok in tokens if not tok.isdigit()]
    return u' '.join(tokens)



In [ ]:


num_partitions = 10 #number of partitions to split dataframe
num_cores = 7 #number of cores on your machine


def processBillLink(link):
    billText = ''
    try:
        req = urllib.request.Request(link, headers={'User-Agent' : "Magic Browser"}) 
        htmlContent = urllib.request.urlopen(req).read()
        
        soup = BeautifulSoup(htmlContent, 'lxml')
        billText = soup.find("pre", {"id": "billTextContainer"}).text
        billText = tokenizeText(billText[:999999].lower()) # trunkate for Spacy
    except Exception:
        print('didnt find', link)
        pass

    return billText

# processBillLink(df_filtered['link'][1])

link_list = df_filtered['link'].tolist()
print(link_list[1])

num_cores = multiprocessing.cpu_count()
print('num_cores', num_cores)
results = Parallel(n_jobs=num_cores)(delayed(processBillLink)(i) for i in tqdm(link_list))


In [3]:
# print(results[1])
# df_filtered['billText'] = results
# df_filtered.to_csv('../data/bill_all.csv')

df_filtered = pd.read_csv('../data/bill_all.csv')
df_filtered.tail()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,index,Unnamed: 0.1.1.1,action_date,action_time,chamber,congress,legis_num,...,link,billText,bill_url,sponsor,sponsor_id,sponsor_party,sponsor_state,sponsor_uri,0,0.1
4057,4057,4057,4057,5158792,5158792,13-Dec-2011,6:59 PM,U.S. House of Representatives,112,H R 2767,...,https://www.congress.gov/bill/112th-congress/h...,112th congress public law u.s government print...,https://api.propublica.org/congress/v1/112/bil...,Gary L. Ackerman,A000022,D,NY,https://api.propublica.org/congress/v1/members...,NaN,NaN
4058,4058,4058,4058,5160455,5160455,14-Dec-2011,6:17 PM,U.S. House of Representatives,112,H R 2105,...,https://www.congress.gov/bill/112th-congress/h...,congressional bills 112th congress u.s governm...,https://api.propublica.org/congress/v1/112/bil...,Gary L. Ackerman,A000022,D,NY,https://api.propublica.org/congress/v1/members...,NaN,NaN
4059,4059,4059,4059,5161291,5161291,14-Dec-2011,6:31 PM,U.S. House of Representatives,112,H R 1264,...,https://www.congress.gov/bill/112th-congress/h...,112th congress public law u.s government print...,https://api.propublica.org/congress/v1/112/bil...,Gary L. Ackerman,A000022,D,NY,https://api.propublica.org/congress/v1/members...,NaN,NaN
4060,4060,4060,4060,5165452,5165452,16-Dec-2011,11:54 AM,U.S. House of Representatives,112,S 278,...,https://www.congress.gov/bill/112th-congress/h...,congressional bills 112th congress u.s governm...,https://api.propublica.org/congress/v1/112/bil...,Gary L. Ackerman,A000022,D,NY,https://api.propublica.org/congress/v1/members...,NaN,NaN
4061,4061,4061,4061,5166702,5166702,16-Dec-2011,2:04 PM,U.S. House of Representatives,112,H R 3672,...,https://www.congress.gov/bill/112th-congress/h...,112th congress public law u.s government print...,https://api.propublica.org/congress/v1/112/bil...,Gary L. Ackerman,A000022,D,NY,https://api.propublica.org/congress/v1/members...,NaN,NaN


In [4]:
## VOTE DETAILS FOR SPONSERSHIP ANALYSIS

api_url = 'https://api.propublica.org/congress/v1/{congress}/house/sessions/{session}/votes/{ID}.json'

api_url_bill = 'https://api.propublica.org/congress/v1/{congress}/bills/{billNum}.json'

def generate_vote_list(row):
    link = api_url.replace('{congress}', str(row['congress']))
    link = link.replace('{session}', row['session'][:1])
    link = link.replace('{ID}', str(row['rollcall_num']))
    return link

def generate_bill_list(row):
    link = api_url_bill.replace('{congress}', str(row['congress']))
    link = link.replace('{billNum}', ''.join(row['legis_num'].split()))
    return link
    

# df_votes['vote_delail_url'] = df_votes.apply(generate_vote_list, axis=1)

df_filtered['bill_url'] = df_filtered.apply(generate_bill_list, axis=1)

df_filtered.bill_url.unique()
 

array(['https://api.propublica.org/congress/v1/110/bills/HRES918.json',
       'https://api.propublica.org/congress/v1/110/bills/HRES912.json',
       'https://api.propublica.org/congress/v1/110/bills/HRES921.json',
       ...,
       'https://api.propublica.org/congress/v1/112/bills/HR1264.json',
       'https://api.propublica.org/congress/v1/112/bills/S278.json',
       'https://api.propublica.org/congress/v1/112/bills/HR3672.json'],
      dtype=object)

In [5]:
df_filtered['bill_url']

0       https://api.propublica.org/congress/v1/110/bil...
1       https://api.propublica.org/congress/v1/110/bil...
2       https://api.propublica.org/congress/v1/110/bil...
3       https://api.propublica.org/congress/v1/110/bil...
4       https://api.propublica.org/congress/v1/110/bil...
5       https://api.propublica.org/congress/v1/110/bil...
6       https://api.propublica.org/congress/v1/110/bil...
7       https://api.propublica.org/congress/v1/110/bil...
8       https://api.propublica.org/congress/v1/110/bil...
9       https://api.propublica.org/congress/v1/110/bil...
10      https://api.propublica.org/congress/v1/110/bil...
11      https://api.propublica.org/congress/v1/110/bil...
12      https://api.propublica.org/congress/v1/110/bil...
13      https://api.propublica.org/congress/v1/110/bil...
14      https://api.propublica.org/congress/v1/110/bil...
15      https://api.propublica.org/congress/v1/110/bil...
16      https://api.propublica.org/congress/v1/110/bil...
17      https:

In [11]:
import requests
# url = df_filtered['bill_url'][1]
print(df_filtered['legis_num'][1])
# print(url)

headers = {'x-api-key': 'cA6VZ1z0NBiXMlt4pthiKrKE4teNc3F9bpJ4Dt7F'}

#  "sponsor": "Doug Collins",
#  "sponsor_id": "C001093",
#  "sponsor_uri": "https://api.propublica.org/congress/v1/members/C001093.json",
#  "sponsor_party": "R",
#  "sponsor_state": "GA",

notFound = 0

def processBillLink(link):
    billText = ''
#     print(link)
    try:
        r = requests.get(url=link, headers=headers)
        result = r.json()
        return {"sponsor": result['results'][0]['sponsor'],
            "sponsor_id": result['results'][0]['sponsor_id'],
            "sponsor_uri": result['results'][0]['sponsor_uri'],
            "sponsor_party": result['results'][0]['sponsor_party'],
            "sponsor_state": result['results'][0]['sponsor_state']}

    except Exception as e:
        print('didnt find', link)
        print(e)
        return {}

processBillLink(df_filtered['bill_url'][1])


H RES 912


{'sponsor': 'Gary L. Ackerman',
 'sponsor_id': 'A000022',
 'sponsor_party': 'D',
 'sponsor_state': 'NY',
 'sponsor_uri': 'https://api.propublica.org/congress/v1/members/A000022.json'}

In [14]:

link_list = df_filtered['bill_url'].tolist()
print(len(link_list))

num_cores = multiprocessing.cpu_count()
print('num_cores', num_cores)
results = Parallel(n_jobs=num_cores)(delayed(processBillLink)(i) for i in tqdm(link_list))

  0%|          | 0/4062 [00:00<?, ?it/s]

4062
num_cores 8


100%|██████████| 4062/4062 [01:24<00:00, 48.04it/s]


In [15]:
print(len(results))
df_temp = pd.DataFrame(results)
df_temp

4062


,sponsor,sponsor_id,sponsor_party,sponsor_state,sponsor_uri
0,Louise M. Slaughter,S000480,D,NY,https://api.propublica.org/congress/v1/members...
1,Gary L. Ackerman,A000022,D,NY,https://api.propublica.org/congress/v1/members...
2,Nydia M. Velázquez,V000081,D,NY,https://api.propublica.org/congress/v1/members...
3,George Miller,M000725,D,CA,https://api.propublica.org/congress/v1/members...
4,Ike Skelton,S000465,D,MO,https://api.propublica.org/congress/v1/members...
5,Maxine Waters,W000187,D,CA,https://api.propublica.org/congress/v1/members...
6,G. K. Butterfield,B001251,D,NC,https://api.propublica.org/congress/v1/members...
7,Howard Coble,C000556,R,NC,https://api.propublica.org/congress/v1/members...
8,John D. Dingell,D000355,D,MI,https://api.propublica.org/congress/v1/members...
9,Dianne Feinstein,F000062,D,CA,https://api.propublica.org/congress/v1/members...


In [27]:
df_temp = pd.DataFrame(results)
# df_filtered = df_filtered.drop(['bill_url', 'sponsor', 'sponsor_id',
#        'sponsor_party', 'sponsor_state', 'sponsor_uri', '0', '0.1', 'sponsor',
#        'sponsor_id', 'sponsor_party', 'sponsor_state', 'sponsor_uri'], axis=1)
df_bills = pd.concat([df_filtered, df_temp], axis=1)
df_bills.to_csv('../data/bill_all.csv')


In [28]:

df_bills.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'index',
       'Unnamed: 0.1.1.1', 'action_date', 'action_time', 'chamber', 'congress',
       'legis_num', 'majority', 'name', 'party', 'role', 'rollcall_num',
       'session', 'state', 'vote', 'vote_desc', 'vote_question', 'vote_result',
       'vote_type', 'link', 'billText', 'sponsor', 'sponsor_id',
       'sponsor_party', 'sponsor_state', 'sponsor_uri'],
      dtype='object')

In [30]:
df_bills['sponsor_id'].unique()

array(['S000480', 'A000022', 'V000081', 'M000725', 'S000465', 'W000187',
       'B001251', 'C000556', 'D000355', 'F000062', 'D000327', 'P000197',
       'B000072', 'O000085', 'G000410', 'G000553', 'H001043', 'S001174',
       'P000149', 'P000034', 'R000487', 'L000397', 'M001148', 'M001138',
       nan, 'H000324', 'C000714', 'R000462', 'H000874', 'L000563',
       'J000126', 'S000097', 'H000206', 'T000260', 'B001229', 'R000568',
       'C001036', 'K000113', 'S001157', 'S001165', 'M001163', 'R000053',
       'V000127', 'D000168', 'Y000062', 'C000380', 'B001231', 'P000589',
       'T000463', 'R000435', 'G000548', 'C001066', 'S000583', 'M000933',
       'L000553', 'M000309', 'E000215', 'H001039', 'B001259', 'R000170',
       'B001258', 'L000274', 'B000589', 'M000312', 'S000749', 'B000574',
       'K000105', 'I000057', 'A000363', 'Y000033', 'B001254', 'G000377',
       'C000984', 'D000136', 'W000738', 'B000410', 'M001167', 'B000490',
       'M001142', 'H000338', 'G000551', 'P000258', 'W0008

In [31]:
df_bills.tail()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,index,Unnamed: 0.1.1.1,action_date,action_time,chamber,congress,legis_num,...,vote_question,vote_result,vote_type,link,billText,sponsor,sponsor_id,sponsor_party,sponsor_state,sponsor_uri
4057,4057,4057,4057,5158792,5158792,13-Dec-2011,6:59 PM,U.S. House of Representatives,112,H R 2767,...,On Motion to Suspend the Rules and Pass,Passed,2/3 RECORDED VOTE,https://www.congress.gov/bill/112th-congress/h...,112th congress public law u.s government print...,John W. Olver,O000085,D,MA,https://api.propublica.org/congress/v1/members...
4058,4058,4058,4058,5160455,5160455,14-Dec-2011,6:17 PM,U.S. House of Representatives,112,H R 2105,...,"On Motion to Suspend the Rules and Pass, as Am...",Passed,2/3 YEA-AND-NAY,https://www.congress.gov/bill/112th-congress/h...,congressional bills 112th congress u.s governm...,Ileana Ros-Lehtinen,R000435,R,FL,https://api.propublica.org/congress/v1/members...
4059,4059,4059,4059,5161291,5161291,14-Dec-2011,6:31 PM,U.S. House of Representatives,112,H R 1264,...,"On Motion to Suspend the Rules and Pass, as Am...",Passed,2/3 YEA-AND-NAY,https://www.congress.gov/bill/112th-congress/h...,112th congress public law u.s government print...,Stephen Fincher,F000458,R,TN,https://api.propublica.org/congress/v1/members...
4060,4060,4060,4060,5165452,5165452,16-Dec-2011,11:54 AM,U.S. House of Representatives,112,S 278,...,"On Motion to Suspend the Rules and Pass, as Am...",Passed,2/3 RECORDED VOTE,https://www.congress.gov/bill/112th-congress/h...,congressional bills 112th congress u.s governm...,Mark Udall,U000038,D,CO,https://api.propublica.org/congress/v1/members...
4061,4061,4061,4061,5166702,5166702,16-Dec-2011,2:04 PM,U.S. House of Representatives,112,H R 3672,...,On Passage,Passed,YEA-AND-NAY,https://www.congress.gov/bill/112th-congress/h...,112th congress public law u.s government print...,Harold Rogers,R000395,R,KY,https://api.propublica.org/congress/v1/members...
